# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), August 26, 2019**

In [1]:
using Pkg
Pkg.activate(".")

Activating environment at `d:\Dev\Julia\DataFrames_Tutorial\Project.toml`


In [2]:
using DataFrames
using BenchmarkTools

## Performance tips

### Access by column number is faster than by name

In [3]:
x = DataFrame(rand(5, 1000))
@btime $x[!, 500];
@btime $x.x500;

  4.199 ns (0 allocations: 0 bytes)
  13.813 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [4]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  82.455 ms (5999020 allocations: 122.06 MiB)


In [5]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  12.327 ms (42 allocations: 30.52 MiB)
  13.004 ms (42 allocations: 30.52 MiB)


In [6]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  12.953 ms (42 allocations: 30.52 MiB)


### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between `DataFrame` and `DataFrame!` (copying vs non-copying data frame creation)

In [7]:
function f1()
    x = DataFrame!([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame!(x) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  21.032 ms (1949736 allocations: 37.42 MiB)
  22.184 ms (1949936 allocations: 45.05 MiB)
  1.585 ms (836 allocations: 7.68 MiB)
  2.646 ms (1036 allocations: 15.32 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [8]:
x = DataFrame(rand(10^6, 5))
y = DataFrame(transpose(1.0:5.0))
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5)) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  13.787 ms (174 allocations: 38.16 MiB)
  359.330 ns (11 allocations: 176 bytes)
  324.348 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [9]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


ArgumentError: ArgumentError: Package StatsBase not found in current path:
- Run `import Pkg; Pkg.add("StatsBase")` to install the StatsBase package.


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [10]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [11]:
@btime by($df, :x, v -> sum(v.y)) # traditional syntax, slow

  300.102 ms (189 allocations: 433.19 MiB)


,x,x1
,Char,Int64
1,'a',2500239
2,'b',2501757
3,'c',2497646
4,'d',2500358


In [12]:
@btime by($df, :x, :y=>sum) # use column selector

  218.191 ms (152 allocations: 356.89 MiB)


,x,y_sum
,Char,Int64
1,'a',2500239
2,'b',2501757
3,'c',2497646
4,'d',2500358


In [13]:
categorical!(df, :x);

In [14]:
@btime by($df, :x, :y=>sum)

  98.036 ms (171 allocations: 152.60 MiB)


,x,y_sum
,Categorical…,Int64
1,'a',2500239
2,'b',2501757
3,'c',2497646
4,'d',2500358


In [15]:
using PooledArrays

In [16]:
df.x = PooledArray{Char}(df.x)

10000000-element PooledArray{Char,UInt8,1,Array{UInt8,1}}:
 'a'
 'b'
 'c'
 'a'
 'b'
 'a'
 'a'
 'a'
 'c'
 'd'
 'd'
 'a'
 'a'
 ⋮  
 'd'
 'b'
 'b'
 'd'
 'a'
 'd'
 'b'
 'b'
 'b'
 'c'
 'd'
 'a'

In [17]:
@btime by($df, :x, :y=>sum)

  94.520 ms (162 allocations: 152.60 MiB)


,x,y_sum
,Char,Int64
1,'a',2500239
2,'b',2501757
3,'c',2497646
4,'d',2500358


### Use views instead of materializing a new DataFrame

In [18]:
x = DataFrame(rand(100, 1000))

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508
2,0.957204,0.273191,0.676637,0.73715,0.477124,0.950504,0.0979649,0.444878
3,0.515305,0.423896,0.253171,0.457358,0.978996,0.794402,0.412566,0.163449
4,0.413562,0.140442,0.127564,0.0370993,0.922239,0.122679,0.56249,0.158148
5,0.747217,0.435913,0.301009,0.0636139,0.396994,0.177713,0.836312,0.831274
6,0.468164,0.300709,0.0108362,0.412532,0.185336,0.494331,0.63317,0.369617
7,0.13405,0.431765,0.537711,0.371646,0.222836,0.358901,0.13515,0.350138
8,0.751183,0.487788,0.278622,0.808394,0.0132744,0.508094,0.400571,0.27485
9,0.721054,0.176363,0.281248,0.408147,0.191549,0.398679,0.0360686,0.78678


In [19]:
@btime $x[1:1, :]

  100.800 μs (1512 allocations: 194.53 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508


In [20]:
@btime $x[1, :]

  23.593 ns (1 allocation: 32 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508


In [21]:
@btime view($x, 1:1, :)

  23.695 ns (1 allocation: 48 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508


In [22]:
@btime $x[1:1, 1:20]

  4.586 μs (56 allocations: 7.36 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508


In [23]:
@btime $x[1, 1:20]

  27.537 ns (2 allocations: 80 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508


In [24]:
@btime view($x, 1:1, 1:20)

  27.235 ns (2 allocations: 96 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.194219,0.0752102,0.924622,0.654874,0.333907,0.084221,0.85674,0.559508
